In [2]:
import pandas as pd

In [78]:
# df = pd.read_excel("transfer_out.xlsx")

# df = pd.read_excel("transfer_out_total.xlsx")
# df = pd.read_excel("transfer_out_chengdu.xlsx")

path ="data/origin_data_for_sankey/transfer_out_hangzhou.xlsx"
path ="data/origin_data_for_sankey/transfer_out_nanjing.xlsx"
path ="data/origin_data_for_sankey/transfer_out_hefei.xlsx"
path ="data/origin_data_for_sankey/transfer_out_wuhan.xlsx"
path ="data/origin_data_for_sankey/transfer_out_quanzhou.xlsx"
path ="data/origin_data_for_sankey/transfer_out_xiamen.xlsx"
path ="data/origin_data_for_sankey/transfer_out_guangzhou.xlsx"

path ="data/origin_data_for_sankey/transfer_out_zimaoqu_zhihang.xlsx"
path ="data/origin_data_for_sankey/transfer_out_fenhangyingyebu_zhihang.xlsx"
path ="data/origin_data_for_sankey/transfer_out_yinhu_zhihang.xlsx"
path ="data/origin_data_for_sankey/transfer_out_qingshan_zhihang.xlsx"
df = pd.read_excel(path)

In [79]:
df

,cate_1,cate_2,amt,ratio
0,购房,购房,1345437,0.014350
1,还款,还款,9967584,0.106313
2,理财,行内理财,335689,0.003580
3,理财,其他理财,1532822,0.016349
4,理财,他行理财,109599,0.001169
5,理财,天天基金理财,37100,0.000396
6,理财,微信理财,384575,0.004102
7,理财,支付宝理财,4711133,0.050249
8,其他,其他,1050625,0.011206
9,取款,取款,6789017,0.072411


In [80]:
df_1 = df.groupby("cate_1", as_index=False)["ratio"].sum().rename(columns={"cate_1":"target", "ratio":"value"})
df_1["source"] = "总资金"

df_2 = df.rename(columns={"cate_1":"source", "cate_2":"target","ratio":"value"})[["source", "target","value"]]

In [81]:
df_1["level"] = 1
df_2["level"] = 2

df_link = pd.concat([df_1, df_2], axis=0)

In [82]:
#df_link.to_csv("transout_sankey.csv", index=False)

In [83]:
colors = { '支付宝理财': "#0066CC",
          '微信': "#00FF00",
          '其他消费': "#66FF00",
          '支付宝': "#3366FF",
          '跨行异名': "#6699CC",
          '微信理财': "#33FF33",
          '美团': "#FFA500",
          '同行同名': "#6B8E23",
          '总资金':"#0099CC",
          '他行理财': "#00CCCC",
          '取款': "#6699FF",
          '第三方转账': "#99CC66",
          '其他理财': "#9999CC",
          '跨行同名': "#99CC00",
          '其他': "#CC9966",
          '京东': "#99CC66",
          '天天基金理财': "#FF9999",
          '理财': "#669999",
          '同行异名': "#6699CC",
          '转账': "#FF9900",
          '行内理财': "#99CC99",
          '购房': "#CCCCFF",
          '还款': "#99CC66",
          '消费': "#33CC66",
         }

In [84]:
## 各个节点的值
pct_1 = df_1.groupby("source")["value"].sum().to_dict()
pct_2 = df_2.groupby("source")["value"].sum().to_dict()
pct_3 = df_2.query("source!=target").groupby("target")["value"].sum().to_dict()

pct = dict()
pct.update(pct_1)
pct.update(pct_2)
pct.update(pct_3)

total = sum(pct_2.values())
pct_new = {n:"{:.2f}%".format(100*(v/total)) if n != "总资金" else "100%" for n, v in pct.items()}


#  构造echart 桑吉图的节点
nodes_ = set(df_1[["source","target"]].values.reshape(-1).tolist()) | set(df_2[["source","target"]].values.reshape(-1).tolist())

nodes = [{"name":n,"itemStyle":{"color":colors[n]}, "value": pct_new[n]} for n in nodes_]

In [85]:
links = df_1.to_dict(orient="records")

links.extend(df_2.query("source!=target").to_dict(orient="records"))

In [86]:
data = {"links": links, "nodes":nodes}

In [87]:
data

{'links': [{'target': '其他',
   'value': 0.0112058741872441,
   'source': '总资金',
   'level': 1},
  {'target': '取款', 'value': 0.0724110604231402, 'source': '总资金', 'level': 1},
  {'target': '消费', 'value': 0.19834255988083188, 'source': '总资金', 'level': 1},
  {'target': '理财', 'value': 0.0758444282820319, 'source': '总资金', 'level': 1},
  {'target': '购房', 'value': 0.0143503131458543, 'source': '总资金', 'level': 1},
  {'target': '转账', 'value': 0.5215323865629565, 'source': '总资金', 'level': 1},
  {'target': '还款', 'value': 0.106313377517942, 'source': '总资金', 'level': 1},
  {'source': '理财', 'target': '行内理财', 'value': 0.00358042945869535, 'level': 2},
  {'source': '理财', 'target': '其他理财', 'value': 0.0163489451359333, 'level': 2},
  {'source': '理财', 'target': '他行理财', 'value': 0.00116897333020609, 'level': 2},
  {'source': '理财',
   'target': '天天基金理财',
   'value': 0.000395705349051048,
   'level': 2},
  {'source': '理财', 'target': '微信理财', 'value': 0.00410184325097862, 'level': 2},
  {'source': '理财', 'targe

In [91]:
def render_option(nodes, links):
    """sankey option
    """
    option ="""
    option = {
        tooltip: {
            trigger: 'item',
            triggerOn: 'mousemove'
        },
        series: [
            {
                type: 'sankey',
                emphasis: {
                    focus: 'adjacency'
                },
                nodeAlign: 'left',
                data: %(nodes)s,
                links: %(links)s,
                label:{formatter:"{b} {c}"},
                lineStyle: {
                    color: 'source',
                    curveness: 0.5,
                }
            }
        ]
    }"""%{"nodes":nodes, "links":links}
    return option

In [92]:
option = render_option(nodes, links)

In [93]:
with open("test.txt", 'w') as f:
    f.write(option)